In [1]:
"""
Un des enjeu sur le DeepLearning et en particulier le Hs NN du WV est d etre capable
de refaire le meme model heteroskedastic_2017.h5 que celui fourni par Sadowski en feb 2021)
Je vais utilier les mêmes libs et le meme training dataset
A Grouazel
June 2021 : reprise du notebook eponyme pour egere transformation sur experience1 : remplacement des X spectres OCN par SLC

"""

'\nUn des enjeu sur le DeepLearning et en particulier le Hs NN du WV est d etre capable\nde refaire le meme model heteroskedastic_2017.h5 que celui fourni par Sadowski en feb 2021)\nJe vais utilier les mêmes libs et le meme training dataset\nA Grouazel\nJune 2021 : reprise du notebook eponyme pour egere transformation sur experience1 : remplacement des X spectres OCN par SLC\n\n'

In [2]:
# Train neural network to predict significant wave height from SAR spectra.
# Train with heteroskedastic regression uncertainty estimates.
# Author: Peter Sadowski, Dec 2020
import os, sys
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' # Needed to avoid cudnn bug.
import numpy as np
import h5py

import tensorflow as tf
from tensorflow.keras.utils import Sequence, plot_model
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

#sys.path = ['../'] + sys.path
sys.path.append('/home1/datahome/agrouaze/git/sar_hs_nn/')
from sarhspredictor.lib.sarhs.generator import SARGenerator
from sarhspredictor.lib.sarhs.heteroskedastic import Gaussian_NLL, Gaussian_MSE

# model definition

In [3]:
def define_model():
    # Low-level features.
    inputs = Input(shape=(72, 60, 2))
    x = Conv2D(64, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = GlobalMaxPooling2D()(x)
    x = Dense(256, activation='relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    cnn = Model(inputs, x)

    # High-level features.
    inp = Input(shape=(32, ))  # 'hsSM', 'hsWW3v2', 'hsALT', 'altID', 'target' -> dropped
    x = Dense(units=256, activation='relu')(inp)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dense(units=256, activation='relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(units=256, activation='relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dropout(0.5)(x)
    ann = Model(inputs=inp, outputs=x)
    
    # Combine
    combinedInput = concatenate([cnn.output, ann.output])
    x = Dense(256, activation="relu")(combinedInput)
    x = Dropout(0.5)(x)
    x = Dense(256, activation="relu", name='penultimate')(x)  
    x = Dropout(0.5)(x)
    x = Dense(2, activation="softplus", name='output')(x)
    model = Model(inputs=[cnn.input, ann.input], outputs=x)
    return model

In [4]:
momo = define_model()
print(momo.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 256)          8448        input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 72, 60, 2)]  0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 256)          65792       dense_2[0][0]                    
____________________________________________________________________________________________

# cell added to get equivalent file sar_hs.h5
 inspired from https://github.com/hawaii-ai/SAR-Wave-Height/blob/master/scripts/create_dataset_from_nc.ipynb
 pas mal de petite modif sur le nom des variables et avec les variables deja assemblees
 
 dataflow:
 training dataset ALT_...nc -> ALT_...processed.nc -> aggregate.h5 -> split by groups .h5

In [50]:
import os, sys, h5py
import numpy as np
import glob
import logging
from importlib import reload
reload(logging)
logging.basicConfig(level=logging.DEBUG)
from netCDF4 import Dataset
import time
from dateutil import rrule
import datetime
import traceback
import xarray
sys.path.append('/home1/datahome/agrouaze/git/sar_hs_nn/sarhspredictor/lib/')
import normalization_inputs_exp1
reload(normalization_inputs_exp1)
out_dd = '/home1/scratch/agrouaze/training_quach_redo_model/exp1/'
in_dd = '/home/datawork-cersat-public/cache/project/mpc-sentinel1/analysis/s1_data_analysis/hs_nn/exp1/training_dataset/v1/'
sta = datetime.datetime(2015,1,1)
sto = datetime.datetime(2016,1,1)
normalization_inputs_exp1.normalize_training_ds(sta,sto,in_dd,out_dd,redo=False)

INFO:root:nb files for month 2015-01-01 00:00:00 : 0
INFO:root:nb files for month 2015-02-01 00:00:00 : 0
INFO:root:nb files for month 2015-03-01 00:00:00 : 0
INFO:root:nb files for month 2015-04-01 00:00:00 : 0
INFO:root:/home1/scratch/agrouaze/training_quach_redo_model/exp1/S1A_training_exp1_Hs_NN_regression_201505.h5 already exists -> skip and continue the month loop
INFO:root:/home1/scratch/agrouaze/training_quach_redo_model/exp1/S1A_training_exp1_Hs_NN_regression_201506.h5 already exists -> skip and continue the month loop
INFO:root:/home1/scratch/agrouaze/training_quach_redo_model/exp1/S1A_training_exp1_Hs_NN_regression_201507.h5 already exists -> skip and continue the month loop
INFO:root:/home1/scratch/agrouaze/training_quach_redo_model/exp1/S1A_training_exp1_Hs_NN_regression_201508.h5 already exists -> skip and continue the month loop
INFO:root:/home1/scratch/agrouaze/training_quach_redo_model/exp1/S1A_training_exp1_Hs_NN_regression_201509.h5 already exists -> skip and continu

# aggregate the monthly processed files

In [56]:
from sarhspredictor.bin import aggregate_monthly_training_files
import glob
import logging
from importlib import reload
reload(logging)
logging.basicConfig(level=logging.DEBUG)
logging.info('ok')
from importlib import reload
reload(aggregate_monthly_training_files)
files_src = sorted(glob.glob(os.path.join(out_dd,'*training*h5')))

print(f'Found {len(files_src)} files.')
print(files_src[0])
# file_dest =  "/mnt/lts/nfs_fs02/sadow_lab/preserve/stopa/sar_hs/data/alt/aggregated_ALT.h5"
# file_dest =  "/mnt/tmp/psadow/sar/aggregated_ALT.h5"
# file_dest = "/mnt/tmp/psadow/sar/aggregated_2019.h5"
# file_dest =  "/mnt/lts/nfs_fs02/sadow_lab/preserve/stopa/sar_hs/data/alt/aggregated_2019.h5"
file_dest = os.path.join('/home1/scratch/agrouaze/training_quach_redo_model/exp1',"aggregated.h5")

# keys = ['timeSAR', 'timeALT', 'lonSAR', 'lonALT', 'latSAR', 'latALT', 'hsALT', 'dx', 'dt', 'nk', 'hsSM', 'incidenceAngle', 'sigma0', 'normalizedVariance', 'S']
# keys = ['timeSAR', 'lonSAR',  'latSAR', 'incidenceAngle', 'sigma0', 'normalizedVariance', 'S']
# keys += ['cspcRe', 'cspcIm']
# keys = ['timeSAR', 'lonSAR',  'latSAR', 'incidenceAngle', 'sigma0', 'normalizedVariance', 'py_S', 'cspcRe', 'cspcIm'] #'py_cspcRe', 'py_cspcIm']
keys = ['timeSAR','timeALT','lonSAR','lonALT','latSAR','latALT','hsALT','dx','dt','nk','hsSM','incidenceAngle','sigma0',
        'normalizedVariance','cspcRe','cspcIm','cwave','todSAR','py_S'] #
t0 = time.time()
aggregate_monthly_training_files.aggregate(files_src,file_dest,keys=keys)
logging.info('done in %1.3f seconds',time.time()-t0)

INFO:root:ok
  0%|          | 0/8 [00:00<?, ?it/s]DEBUG:root:date parsed: 201505


Found 8 files.
/home1/scratch/agrouaze/training_quach_redo_model/exp1/S1A_training_exp1_Hs_NN_regression_201505.h5
['S1A', 'training', 'exp1', 'Hs', 'NN', 'regression', '201505', 'h5']


 12%|█▎        | 1/8 [00:00<00:05,  1.22it/s]DEBUG:root:date parsed: 201506


['S1A', 'training', 'exp1', 'Hs', 'NN', 'regression', '201506', 'h5']


 25%|██▌       | 2/8 [00:02<00:05,  1.02it/s]DEBUG:root:date parsed: 201507


...

 75%|███████▌  | 6/8 [00:11<00:04,  2.09s/it]DEBUG:root:date parsed: 201511


['S1A', 'training', 'exp1', 'Hs', 'NN', 'regression', '201511', 'h5']


 88%|████████▊ | 7/8 [00:13<00:02,  2.19s/it]DEBUG:root:date parsed: 201512


['S1A', 'training', 'exp1', 'Hs', 'NN', 'regression', '201512', 'h5']


100%|██████████| 8/8 [00:17<00:00,  2.13s/it]
INFO:root:done in 17.026 seconds


# split by groups

In [71]:
# the training dataset must be separated into sub groups
# long long task (about 30min)
import split_aggregated_into_groups
from importlib import reload
reload(split_aggregated_into_groups)
file_src2 = os.path.join('/home1/scratch/agrouaze/training_quach_redo_model/exp1',"aggregated.h5")
print('source ',file_src2)
file_dest2 = '/home1/scratch/agrouaze/training_quach_redo_model/aggregated_grouped_final_exp1.h5'
if os.path.exists(file_dest2):
    os.remove(file_dest2)
t0 = time.time()
split_aggregated_into_groups.split_aggregated_ds_v2(file_src2,file_dest2,test2015=True)
logging.info('done in %1.3f seconds',time.time()-t0)

source  /home1/scratch/agrouaze/training_quach_redo_model/exp1/aggregated.h5


INFO:root:group_name: group_1 months: [1, 2, 3, 4, 5, 6]
INFO:root:month val :(19118,) int64
INFO:root:fs[cspcRe] : (19118, 1, 60, 72)


k cspcIm
k cspcRe
k cwave
k dt
k dx
k hsALT
k hsSM
k incidenceAngle
k latALT
k latSAR
k lonALT
k lonSAR
k month
k nk
k normalizedVariance
k py_S
k satellite
k sigma0
k timeALT
k timeSAR
k todSAR
k year
start creating the final .h5 file
indices 1 (19118,) 0
indices 2 (19118,) 0
indices 3 (19118,) 0
indices 4 (19118,) 0
indices 5 (19118,) 700
indices 6 (19118,) 2198
Found 2198 events from months:  [1, 2, 3, 4, 5, 6]
timeSAR (19118, 1)


INFO:root:group_name: group_2 months: [7, 8, 9]
INFO:root:month val :(19118,) int64


Done with [1, 2, 3, 4, 5, 6]
indices 7 (19118,) 1875
indices 8 (19118,) 5317
indices 9 (19118,) 7713
Found 7713 events from months:  [7, 8, 9]


...

Done with [7, 8, 9]
indices 10 (19118,) 3130
indices 11 (19118,) 5833
indices 12 (19118,) 9207
Found 9207 events from months:  [10, 11, 12]


INFO:root:fs[cspcRe] : (19118, 1, 60, 72)


timeSAR (19118, 1)


INFO:root:done in 15.839 seconds


Done with [10, 11, 12]
Done


# training keras

In [72]:
# Train
from sarhspredictor.config import model_IFR_replication_quach2020_sadowski_release_5feb2021
#file_model = '/home1/scratch/agrouaze/heteroskedastic_2017_agrouaze.h5'
file_model = model_IFR_replication_quach2020_sadowski_release_5feb2021 #
print(file_model)
model = define_model()
model.compile(loss=Gaussian_NLL, optimizer=Adam(lr=0.0001), metrics=[Gaussian_MSE])

# Dataset
batch_size = 128
epochs = 123
#filename = '../../data/alt/sar_hs.h5'
#filename = '/mnt/tmp/psadow/sar/sar_hs.h5'
filename = file_dest2
print(file_dest2)
train = SARGenerator(filename=filename, 
                     subgroups=['group_1', 'group_2'], 
                     batch_size=batch_size)
valid = SARGenerator(filename=filename, subgroups=['group_3'], batch_size=batch_size)
# filename = '/mnt/tmp/psadow/sar/sar_hs.h5'
# epochs = 25
# train = SARGenerator(filename=filename, 
#                      subgroups=['2015_2016', '2017', '2018'], # Train on all data without early stopping.
#                      batch_size=batch_size)

# Callbacks
# This LR schedule is slower than in the paper.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=1) 
check = ModelCheckpoint(file_model, monitor='val_loss', verbose=0,
                        save_best_only=True, save_weights_only=False,
                        mode='auto', save_freq='epoch')
stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, 
                     mode='auto', baseline=None, restore_best_weights=False)
clbks = [reduce_lr, check, stop]

history = model.fit(train,
                    epochs=epochs,
                    validation_data=valid,
                    callbacks=clbks,
                    verbose=1)

/home1/datahome/agrouaze/sources/sentinel1/hs_total/validation_quach2020/heteroskedastic_2017_agrouaze.h5
/home1/scratch/agrouaze/training_quach_redo_model/aggregated_grouped_final_exp1.h5
Epoch 1/123
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute '

KeyboardInterrupt: 